In [ ]:
import numpy as np
import matplotlib.pylab as plt

# 4. 신경망 학습

<code>학습</code>: 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것

## 4.2 손실 함수

### 4.2.1 평균 제곱 오차

<code>평균 제곱 오차(mean squared error, MSE)</code>

$$ E = \frac{1}{n} \sum_{k}(y_k - t_k)^2 $$

각 원소의 출력(추정 값)과 정답 레이블(참 값)의 차를 제곱하고 모두 합한 후 평균  

※ 책에는 $\frac{1}{n}$이 아니라 $\frac{1}{2}$로 나와있는데 왜? <span style='color:pink'>!!!질문!!!</span>

$y_k$: 신경망의 출력(신경망이 추정한 값)  
$t_k$: 정답 레이블  
$k$: 데이터의 차원 수

In [21]:
def mean_squared_error(y, t):
    return np.sum((y-t)**2) / len(t)

### 4.2.2 교차 엔트로피 오차

<code>교차 엔트로피 오차(cross entropy error, CEE)</code>

$$ E = -\sum_{k} t_{k} \log{y_{k}} $$

여기서 $\log$는 밑이 $e$인 자연로그  

$y_k$: 신경망의 출력  
$t_k$: 정답 레이블  

$t_k$는 정답에 해당하는 인덱스의 원소만 1이고 나머지는 0 (원핫인코딩)  
→ $t_k=0$일 때는 모두 무시 가능하고, $t_k=1$일 때(정답일 때)만 자연로그 계산

예를 들어 정답 레이블은 '2'가 정답이라 하고 이때의 신경망 출력 $y_k$가 $0.6$이라면 교차 엔트로피 오차는 $-\log0.6 = 0.51$  
같은 조건에서 신경망 출력 $y_k$가 $0.1$이라면 교차 엔트로피 오차는 $-\log0.1 = 2.3$  
(상대적으로 제대로 예측한 경우(신경망 출력이 $0.6$인 경우) loss가 작고  
 상대적으로 잘못 예측한 경우(신경망 출력이 $0.1$인 경우) loss가 큼)

<img src="https://user-images.githubusercontent.com/77653353/192322965-5d57ab8b-a5b1-4b2f-a79c-c9a0cae2b55d.png">

In [22]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

아주 작은 값인 delta를 더하여 절대 0이 되지 않도록, 즉 마이너스 무한대가 발생하지 않도록 한 것

### 4.2.3 미니배치 학습

이제 데이터 하나가 아닌 훈련 데이터 모두에 대한 손실 함수의 합을 구하는 방법을 생각해보자

훈련 데이터 모두에 대한 교차 엔트로피 오차

$$ E = - \frac{1}{N}\sum_{n}\sum_{k}t_{nk}\log y_{nk} $$

데이터가 $N$개라면 $t_{nk}$는 $n$번째 데이터의 $k$차원 째의 값을 의미  

$y_{nk}$: 신경망의 출력  
$t_{nk}$: 정답 레이블  

수식이 좀 복잡해 보이지만 데이터 하나에 대한 손실함수를 단순히 $N$개의 데이터로 확장했을 뿐임  
마지막에 $N$으로 나누어 정규화하여 '평균 손실 함수'를 구하는 것

수백만, 수천만개의 데이터를 일일이 계산하기에는 쉽지 않기 때문에  
훈련 데이터로부터 일부만 골라 학습을 수행하는데 이 일부를 <code>미니배치(mini-batch)</code>

### 4.2.4 (배치용) 교차 엔트로피 오차 구현하기

<span style='color:pink'>!!!질문!!!</span> ?

정답 레이블이 원-핫 인코딩인 경우

In [10]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y)) / batch_size

정답 레이블이 원-핫 인코딩이 아니라 '$2$'나 '$7$' 등의 숫자 레이블로 주어지는 경우

In [1]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y[np.arange(batch_size), t])) / batch_size

정답 레이블이 원-핫 인코딩인 경우, t가 0인 원소는 교차 엔트로피 오차도 0이므로 그 계산을 무시할 수 있음  
  
정답 레이블이 숫자 레이블로 주어지는 경우, np.log(y[np.arange(batch_szie), t])  
np.arange(batch_size)는 0부터 batch_size-1까지 배열을 생성함  
(ex. batch_size=5라면, [0,1,2,3,4]라는 넘파이 배열 생성, t에는 레이블이 [2,7,0,9,4]와 같이 저장됨)  

### 4.2.5 왜 손실 함수를 설정하는가?

우리의 궁극적인 목적은 높은 '정확도'를 끌어내는 매개변수 값을 찾는 것!  
그렇다면 왜 '정확도'라는 지표를 두고 손실 함수의 값을 거쳐갈까?

신경망 학습에서의 <code>미분</code>의 역할에 주목!  
가령 가상의 신경망이 있고, 그 신경망의 어느 한 가중치 매개변수에 주목한다고 해보자  
이때 그 가중치 매개변수의 손실 함수의 미분이란 '가중치 매개변수의 값을 아주 조금 변화시켰을 때, 손실 함수가 어떻게 변하나'라는 의미  
만약 이 미분 값이 음수면, 그 가중치 매개변수를 양의 방향으로 변화시켜 손실 함수의 값을 줄일 수 있고  
만약 이 미분 값이 양수면, 그 가중치 매개변수를 음의 방향으로 변화시켜 손실 함수의 값을 줄일 수 있음  
만약 이 미분 값이 0이면, 가중치 매개변수를 어느 쪽으로 움직여도 손실 함수의 값은 달라지지 않음

계단 함수의 미분은 대부분의 장소(0 이외의 곳)에서 0 → 계단 함수를 활성화 함수로 이용하면 손실함수를 지표로 삼는 게 아무 의미 없어짐  
매개변수의 작은 변화가 주는 파장을 계단 함수가 말살하여 손실 함수의 값에는 아무런 변화가 나타나지 않기 때문

## 4.3 수치 미분

### 4.3.1 미분

<code>미분</code>: 특정 순간의 변화량  
$x$의 작은 변화가 함수$f(x)$를 얼마나 변화시키느냐를 의미

$$ \frac{df(x)}{dx} = \lim_{h→0}\frac{f(x+h)-f(x)}{h} $$

수치 미분(numerical differentiation): 아주 작은 차분(임의 두 점에서의 함수 값들의 차이)으로 미분을 구하는 것  
해석적 미분(analytic differentiation): 수식을 전개해 미분을 구하는 것 ex.$y = x^2$의 미분은 $\frac{dy}{dx} = 2x$

### 4.3.3 편미분

$$ f(x_0, x_1) = x_0^2 + x_1^2 $$

인수들의 제곱 합을 계산하는 단순한 식이지만, 변수가 2개

<code>편미분</code>: 변수가 여럿인 함수에 대한 미분, $\frac{\partial{f}}{\partial{x_0}}$이나 $\frac{\partial{f}}{\partial{x_1}}$

## 4.4 기울기

In [ ]:
dkljflskdjflk한글로